Cat or Dog Neural Network

In [115]:
#load modules
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import preprocessing
import tensorflow as tf
import os
from PIL import Image
import splitfolders

Image Processing

In [105]:
#goes through each image and checks if it is non-corrupt, deletes if it is corrupted
folder_path = 'PetImages'
extensions = []
for fldr in os.listdir(folder_path):
    sub_folder_path = os.path.join(folder_path, fldr)
    for filee in os.listdir(sub_folder_path):
        file_path = os.path.join(sub_folder_path, filee)
        print('** Path: {}  **'.format(file_path), end="\r", flush=True)
        try:
            im = Image.open(file_path)
            rgb_im = im.convert('RGB')
            if filee.split('.')[1] not in extensions:
                extensions.append(filee.split('.')[1])
        except: 
            os.remove(file_path)

Using the above section of code, cats/666.jpg and dogs/11702.jpg were identified as corrupt and deleted

In [116]:
#splits each folder into .8 training, .1 validation, and .1 testing
splitfolders.ratio("PetImages", output="PetImagesSplit",
    seed=1337, ratio=(.8, .1, .1), group_prefix=None, move=False)

Copying files: 24998 files [00:19, 1253.47 files/s]


In [106]:
training_data_generator = ImageDataGenerator(
        #Rescale the image by 1/255 to normalize the pixel values
        rescale=1.0/255,
        #Randomly increase or decrease the size of the image by up to 20%
        zoom_range=0.2, 
        #Randomly rotate the image between -15,15 degrees
        rotation_range=15, 
        #Shift the image along its width by up to +/- 5%
        width_shift_range=0.05, 
        #Shift the image along its height by up to +/- 5%
        height_shift_range=0.05 
)

In [118]:
#variables being fed to training data generator
train_directory = "PetImagesSplit/train/" #path to the folder containing the images to train
val_directory = "PetImagesSplit/val/" #path to the folder containing the images to val
test_directory = "PetImagesSplit/test/" #path to the folder containing the images to test
class_mode = "categorical"
color_mode = "rgb" 
target_size = (256, 256) #resizes each image to 256x256
batch_size = 16

In [119]:
training_iterator = training_data_generator.flow_from_directory(train_directory, class_mode =class_mode ,color_mode =color_mode ,target_size = target_size, batch_size = batch_size)

Found 19998 images belonging to 2 classes.


Mode Building

In [120]:
model = tf.keras.Sequential()

In [121]:
model.add(tf.keras.Input(shape=(256, 256, 3)))
model.add(tf.keras.layers.Conv2D(2, 5, strides=3, activation="relu")) 
model.add(tf.keras.layers.MaxPooling2D(
    pool_size=(5, 5), strides=(5,5)))
model.add(tf.keras.layers.Conv2D(4, 3, strides=1, activation="relu")) 
model.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2,2), strides=(2,2)))
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(2,activation="softmax"))

In [122]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 84, 84, 2)         152       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 16, 16, 2)        0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 14, 14, 4)         76        
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 7, 7, 4)          0         
 2D)                                                             
                                                                 
 flatten_9 (Flatten)         (None, 196)               0         
                                                                 
 dense_19 (Dense)            (None, 2)               

In [123]:
#model compilation
#model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.AUC()])

In [124]:
validation_data_generator = ImageDataGenerator( rescale=1./255)
validation_iterator = validation_data_generator.flow_from_directory(val_directory, class_mode =class_mode ,color_mode =color_mode ,target_size = target_size, batch_size = batch_size)

Found 2498 images belonging to 2 classes.


In [125]:
model.fit(
       training_iterator,
       steps_per_epoch=training_iterator.samples/batch_size,
       epochs=2,
       validation_data=validation_iterator,
       validation_steps=validation_iterator.samples/batch_size)

Epoch 1/2
 247/1249 [====>.........................] - ETA: 3:24 - loss: 0.6936 - categorical_accuracy: 0.5023 - auc_6: 0.5061

c:\Users\Wiggles\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:819: UserWarning: Truncated File Read
  warnings.warn(str(msg))


1249/1249 [==============================] - 248s 198ms/step - loss: 0.6935 - categorical_accuracy: 0.5021 - auc_6: 0.5000 - val_loss: 0.6932 - val_categorical_accuracy: 0.5004 - val_auc_6: 0.5008
Epoch 2/2
1249/1249 [==============================] - 186s 149ms/step - loss: 0.6936 - categorical_accuracy: 0.4960 - auc_6: 0.4946 - val_loss: 0.6933 - val_categorical_accuracy: 0.5000 - val_auc_6: 0.5002
